In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('LR').getOrCreate()

In [0]:
# Loading library for Linear Regression
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

### Label Column has numerical value while features column has vector

In [0]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [0]:
lrModel = lr.fit(training)

In [0]:
lrModel.coefficients

Out[11]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[12]: 0.14228558260358093

In [0]:
training_summary = lrModel.summary

In [0]:
training_summary.r2

Out[14]: 0.027839179518600154

In [0]:
training_summary.rootMeanSquaredError

Out[15]: 10.16309157133015

## Train-Test Split

In [0]:
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [0]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                353|
|   mean|0.15361078655886243|
| stddev| 10.645895844884725|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                148|
|   mean| 0.5032208936784525|
| stddev|  9.519998653473852|
|    min|-28.571478869743427|
|    max| 22.923352376063196|
+-------+-------------------+



In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -28.63260171816476|
| -21.42552258600558|
| -20.57640754753478|
|-17.750338683941795|
|-17.939026976524065|
|-16.086017331822564|
|-19.456897854015853|
|-13.917051383595723|
|-13.458605429282228|
|-10.086429495610194|
|-12.487191906906673|
| -8.834906515473104|
| -9.402999842265192|
|-11.551138280019792|
|-11.374882746241248|
|-13.813775030576004|
|-10.477284056061908|
|  -9.08633658710171|
|-10.260334457900084|
|   -7.8741657627626|
+-------------------+
only showing top 20 rows



In [0]:
test_results.rootMeanSquaredError

Out[26]: 9.577520174432792

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



### Deploying model on unlabeled data

In [0]:
predictions = correct_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| 0.06112284842133493|
|(10,[0,1,2,3,4,5,...|  1.5525315479371757|
|(10,[0,1,2,3,4,5,...|  1.7304850746362002|
|(10,[0,1,2,3,4,5,...|   0.423617951265845|
|(10,[0,1,2,3,4,5,...|  0.8736273506480512|
|(10,[0,1,2,3,4,5,...| -0.6061896894885429|
|(10,[0,1,2,3,4,5,...|  3.6762128213925513|
|(10,[0,1,2,3,4,5,...| -0.8457068693354038|
|(10,[0,1,2,3,4,5,...| -0.4084824658765399|
|(10,[0,1,2,3,4,5,...|  -3.066906110755336|
|(10,[0,1,2,3,4,5,...| -0.0713838819495157|
|(10,[0,1,2,3,4,5,...| -3.6327498655597554|
|(10,[0,1,2,3,4,5,...|  -3.007946560574969|
|(10,[0,1,2,3,4,5,...|-0.35384862265532296|
|(10,[0,1,2,3,4,5,...|-0.48246761918817743|
|(10,[0,1,2,3,4,5,...|   2.197999765560377|
|(10,[0,1,2,3,4,5,...|-0.12284628584712447|
|(10,[0,1,2,3,4,5,...|  -1.402820536271188|
|(10,[0,1,2,3,4,5,...|-0.03533147812983538|
|(10,[0,1,2,3,4,5,...|  -2.35927

Metrics for Continuous Values :
- Mean Absolute Error (Average Error - Absolute so that postives don't cancel negatives)
- Mean Squared Error (Larger Error are noted more than with MAE - punishing effect to larger error)
- Root Mean Square Error (same unit as Labels)
- R Squared Values (Statistical property of the model) | Coefficient of Determination | Between 0 to 1 | How much variance the model accounts for

## Another Example - Analysing Ecommerce Data from a Company's Website and Mobile App

### Predict - Customer total amount expenditure

In [0]:
sparklr = SparkSession.builder.appName('lrex').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
data = sparklr.read.csv('/FileStore/tables/Ecommerce_Customers.csv',inferSchema=True, header=True)

In [0]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [0]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
from pyspark.ml.linalg import Vectors

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

Out[39]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                            outputCol='features')

In [0]:
output = assembler.transform(data)

In [0]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
output.head(1)

Out[43]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
final_data = output.select(['features','Yearly Amount Spent'])

In [0]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                360|
|   mean|  498.9402307005587|
| stddev|  78.58866394817595|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                140|
|   mean|    500.27525769353|
| stddev|  81.43050432026486|
|    min|  282.4712457199145|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
lr2 = LinearRegression(labelCol='Yearly Amount Spent')

In [0]:
lr2_model = lr2.fit(train_data)

In [0]:
test_results2 = lr2_model.evaluate(test_data)

In [0]:
test_results2.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -6.123908020445526|
|  6.916752087507803|
|  3.904943072307674|
| -13.03397466969426|
| 22.361771677481556|
| -4.345750048526952|
|-7.5412043929035235|
| -9.762929239425546|
|-17.761759759143388|
|-14.344596999516853|
| 17.772599020007192|
| -26.59263181297922|
|-3.4718201314396993|
| 0.5807342874979895|
|  -2.37332528348918|
| 11.448780025377175|
|  8.175680085409681|
| 3.3801623087926487|
| 17.071276154764746|
|  4.553479006617863|
+-------------------+
only showing top 20 rows



In [0]:
test_results2.rootMeanSquaredError

Out[53]: 10.533178562754303

In [0]:
test_results2.r2

Out[54]: 0.9831477505271641

### Model explains 98 percent of the variance

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
predictions = lr2_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|288.59515374036005|
|[30.9716756438877...| 487.7218576693849|
|[31.0472221394875...| 388.5924561167137|
|[31.0662181616375...| 461.9672678773686|
|[31.2834474760581...| 569.4193177481859|
|[31.4252268808548...| 535.1124687032889|
|[31.4474464941278...|426.14394648812754|
|[31.5261978982398...|418.85745543176336|
|[31.5702008293202...| 563.7072519005483|
|[31.5741380228732...| 558.7538691601037|
|[31.6005122003032...|461.40025247108974|
|[31.6739155032749...| 502.3176997228604|
|[31.8124825597242...| 396.2821651152369|
|[31.8293464559211...|  384.571603700477|
|[31.8627411090001...| 558.6714664575359|
|[31.9262720263601...| 380.7561534189492|
|[31.9549038566348...| 431.8221998545173|
|[32.0123007682454...| 489.5648907571656|
|[32.0180740106320...|340.71183459055055|
|[32.0542618511847...| 557.3211786623651|
+--------------------+------------